In [1]:
import pandas as pd
import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections

Using TensorFlow backend.


# Extract data 

In [2]:
lung_data_TANT = pd.read_csv("Data/gene_expression_TANT.csv")

In [3]:
lung_data_TANT

,Unnamed: 0,AAACCTGAGTGTTTGC-1,AAACCTGGTAATAGCA-1,AAACCTGTCATCATTC-1,AAACGGGCAGCCACCA-1,AAAGATGTCCGAGCCA-1,AAAGCAAAGCACAGGT-1,AAATGCCAGCCAACAG-1,AACACGTCAGCCTTGG-1,AACCATGAGCGGATCA-1,...,TTTCCTCGTTGTCGCG-1,TTTCCTCTCAGTTTGG-1,TTTGGTTCACATGACT-1,TTTGGTTCACCAGTTA-1,TTTGGTTCATTGGGCC-1,TTTGGTTGTCTCCATC-1,TTTGGTTTCTCCAGGG-1,TTTGTCAAGATCTGAA-1,TTTGTCAAGTGGAGAA-1,TTTGTCACATGCCACG-1
0,ENSG00000279457,-0.539817,1.297965,1.212418,-0.546885,-0.539483,1.304409,0.908951,-0.539113,-0.535508,...,-0.533896,-0.531435,-0.439839,-0.475650,-0.486416,-0.536632,0.425223,2.745316,-0.502533,-0.536619
1,ENSG00000228463,-0.218309,2.582124,-0.332286,-0.513947,-0.275251,-0.409002,-0.602251,-0.278380,5.632476,...,-0.389953,-0.291301,1.364185,-0.512868,-0.400430,-0.237653,0.977453,-0.314023,1.722685,-0.249837
2,ENSG00000237491,-0.177760,-0.182360,-0.184003,-0.154750,-0.174180,-0.176651,-0.168250,-0.174159,-0.176517,...,-0.169436,-0.177165,-0.205801,-0.190603,-0.192596,-0.178086,-0.197115,-0.174260,-0.180241,-0.177291
3,ENSG00000225880,-0.142332,-0.116706,-0.117489,-0.084303,-0.131016,-0.102656,-0.063639,-0.130365,-0.131714,...,-0.107818,-0.127173,-0.067323,-0.078666,5.053023,-0.138235,-0.088054,-0.122899,-0.090336,-0.135819
4,ENSG00000230368,-0.216571,-0.230431,-0.244483,0.023304,-0.177152,-0.171232,-0.067329,-0.176393,9.731362,...,-0.117410,-0.197862,-0.375430,-0.262486,-0.299883,-0.215468,-0.323933,-0.170372,-0.189082,-0.206799
5,ENSG00000223764,-0.100449,-0.132133,-0.136145,-0.078883,-0.098775,-0.126526,-0.131765,-0.099197,-0.104785,...,-0.102590,-0.109935,-0.233883,-0.183115,-0.171638,-0.104372,4.514345,-0.104968,-0.145607,-0.103928
6,ENSG00000187634,-0.253855,2.876000,-0.327195,-0.168340,-0.243373,-0.295286,-0.286095,-0.244040,-0.258477,...,-0.239622,-0.268518,-0.538079,1.242287,-0.405227,-0.261146,2.238930,-0.253951,-0.335032,-0.258696
7,ENSG00000188976,-0.341993,2.085098,-0.408707,-0.410614,-0.357478,-0.422833,1.462533,-0.358799,-0.361431,...,-0.395245,-0.372484,1.073427,1.696691,-0.461539,-0.351363,-0.501009,-0.374730,-0.457064,-0.354597
8,ENSG00000187961,-0.119773,-0.096369,-0.096468,-0.078314,-0.111375,-0.086379,-0.056061,-0.110828,-0.111163,...,-0.093263,-0.107073,-0.046233,-0.061138,-0.080179,-0.116173,-0.066882,-0.104437,-0.074650,-0.114390
9,ENSG00000188290,-0.533981,-0.563712,0.811385,-0.162129,-0.473381,-0.471864,0.833736,-0.472337,-0.504925,...,-0.383258,-0.508005,0.776358,-0.626431,3.068812,-0.533362,-0.723244,-0.464714,-0.504270,-0.520014


In [3]:
somatic_mutation_df = pd.read_csv("Data/Cosmic.tsv", sep='\t')

somatic_mutation_df = somatic_mutation_df[
    (somatic_mutation_df['Mutation somatic status'] == 'Confirmed somatic variant') & 
    (somatic_mutation_df['Genome-wide screen'] == 'y')]

somatic_mutation_df = somatic_mutation_df.drop(['Accession Number', 'Gene CDS length', 
                                                'Mutation CDS', 'Mutation AA', 'Mutation Description', 
                                                'Mutation zygosity', 'LOH', 'GRCh', 'Mutation genome position', 
                                                'Mutation strand', 'SNP', 'FATHMM score', 
                                                'Mutation somatic status', 'Genome-wide screen'], axis=1)

In [5]:

most_common_genes = collections.Counter(somatic_mutation_df['Gene name']).most_common(100)
gene_columns = np.array([x[0] for x in most_common_genes])


In [6]:
filtered_mutation_df = somatic_mutation_df[
    (somatic_mutation_df['Gene name'].isin(gene_columns))]

In [7]:
sample_genes = pd.crosstab(somatic_mutation_df['Sample name'], filtered_mutation_df['Gene name'])
sample_primary_histology = pd.crosstab(somatic_mutation_df['Sample name'], 
                                       filtered_mutation_df['Primary histology']).clip(0,1)
sample_primary_histology = sample_primary_histology.drop(['adrenal_cortical_neoplasm', 'ganglioneuroblastoma', 
                                                          'in_situ_neoplasm', 'leiomyosarcoma', 'liposarcoma', 
                                                          'malignant_peripheral_nerve_sheath_tumour', 'neurofibroma', 
                                                          'olfactory_neuroblastoma', 'other', 'paraganglioma', 
                                                          'rhabdoid_tumour', 'rhabdomyosarcoma', 'synovial_sarcoma', 
                                                          'thymoma', 'thymic_carcinoma'], axis=1)

sample_primary_histology = sample_primary_histology.drop(['NS'], axis=1)

In [8]:
data = sample_genes.values
labels = sample_primary_histology['carcinoma'].values

In [ ]:
def get_data_and_labels(max_num_genes, histology):
    most_common_genes = collections.Counter(somatic_mutation_df['Gene name']).most_common(max_num_genes)
    gene_columns = np.array([x[0] for x in most_common_genes])
    filtered_mutation_df = somatic_mutation_df[
    (somatic_mutation_df['Gene name'].isin(gene_columns))]
    sample_genes = pd.crosstab(somatic_mutation_df['Sample name'], filtered_mutation_df['Gene name'])
    sample_primary_histology = pd.crosstab(somatic_mutation_df['Sample name'], 
                                       filtered_mutation_df['Primary histology']).clip(0,1)
    
    data = sample_genes.values
    labels = sample_primary_histology[histology].values
    
    return data, labels
    

In [ ]:
num_genes = np.array[100, 500, 1000, 1500, 2000]

for max_num_genes in num_genes:
    data, labels = get_data_and_labels(max_num_genes, 'carcinoma')
    results = SVM_classifier_multiple_scoring(data, labels)
    write_results_to_file('svm-carcinoma.txt', results)
    
    

In [9]:
unique, counts = np.unique(labels, return_counts=True)
counts

array([ 4262, 11900])

In [10]:
sample_primary_histology.columns.values

array(['Ewings_sarcoma-peripheral_primitive_neuroectodermal_tumour',
       'Wilms_tumour', 'adenoma', 'adrenal_cortical_adenoma',
       'adrenal_cortical_carcinoma', 'angiosarcoma',
       'atypical_teratoid-rhabdoid_tumour', 'carcinoid-endocrine_tumour',
       'carcinoma', 'chondrosarcoma', 'craniopharyngioma',
       'germ_cell_tumour', 'glioma', 'haemangioblastoma',
       'haematopoietic_neoplasm', 'lymphoid_neoplasm',
       'malignant_melanoma', 'meningioma', 'mesothelioma', 'neuroblastoma',
       'osteosarcoma', 'pancreatic_intraepithelial_neoplasia_(PanIN)',
       'pheochromocytoma',
       'primitive_neuroectodermal_tumour-medulloblastoma',
       'pulmonary_blastoma'], dtype=object)

# Classifiers 

In [ ]:
from sklearn import svm, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# SVM classifier

In [ ]:
def SVM_classifier(train_data, train_labels, test_data, test_labels):
    SVM = svm.SVC()
    
    g_range = 2. ** np.arange(-15, -5, step=1)
    C_range = 2. ** np.arange(-5, 10, step=1)

    parameters = {'kernel':('rbf', 'linear'), 'gamma':g_range, 'C':C_range}
    
    grid = GridSearchCV(SVM, parameters, n_jobs=4, verbose=2)
    grid.fit(train_data, train_labels)
    print grid.best_params_
    
    test_pred = grid.predict(test_data)
    
    return metrics.accuracy_score(test_labels, test_predictions)

In [ ]:
def SVM_cross_validation(data, labels):
    stratified_k_fold = StratifiedKFold(n_splits=10)
    scores = []
    
    for train_index, test_index in stratified_k_fold.split(data, labels):
        train_data, test_data = data[train_index], data[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]
        score = SVM_classifier(train_data, train_labels, test_data, test_labels)
        print score
        scores.append(score)
        
    scores = np.array(scores)
    print np.mean(scores)
    print np.stdev(scores)
        
    return scores, np.mean(scores), np.stdev(scores)
    

In [ ]:
SVM_cross_validation(data, labels)

# Multiple scores for SVMs

In [ ]:
def SVM_classifier_multiple_scoring(data, labels):
    scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']
    
    SVM = svm.SVC(kernel='rbf')
    
    g_range = 2. ** np.arange(-10, 5, step=1)
    C_range = 2. ** np.arange(-5, 10, step=1)

    parameters = {'kernel':('rbf', 'linear'), 'gamma':g_range, 'C':C_range}
    
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
    grid = GridSearchCV(SVM, parameters, n_jobs=1, cv=kfold)
    cv_results = cross_validate(grid, data, labels, scoring=scoring, cv=kfold, verbose=2)
    
    return cv_results

#cv_results = SVM_classifier_multiple_scoring(data, labels)
#cv_results

# Random forest classifier

In [ ]:
def random_forest_classifier(train_data, train_labels, test_data, test_labels):
    random_forest = RandomForestClassifier(max_depth=None, min_samples_split=2, random_state=0)
    
    n_estimators = np.arange(10, 100, step=1)
    parameters = {'n_estimators':n_estimators, 'criterion':('gini', 'entropy')}
    
    grid = GridSearchCV(estimator=random_forest, param_grid=parameters, n_jobs=4, verbose=2)
    grid.fit(train_data, train_labels)
    print grid.best_params_
    
    test_pred = grid.predict(test_data)
    
    return metrics.accuracy_score(test_labels, test_pred)

In [ ]:
def RF_cross_validation(data, labels):
    stratified_k_fold = StratifiedKFold(n_splits=10)
    scores = []
    
    for train_index, test_index in stratified_k_fold.split(data, labels):
        train_data, test_data = data[train_index], data[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]
        score = random_forest_classifier(train_data, train_labels, test_data, test_labels)
        print score
        scores.append(score)
        
    scores = np.array(scores)
    print np.mean(scores)
    print np.std(scores)
        
    return scores, np.mean(scores), np.std(scores)

In [ ]:
rf_scores, rf_mean, rf_stdev = RF_cross_validation(data, labels)

# {'n_estimators': 95, 'criterion': 'gini'}

# Multiple scores for Random Forests

In [ ]:
def random_forests_classifier_multiple_scoring(data, labels):
    scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']
    
    random_forest = RandomForestClassifier(max_depth=None, min_samples_split=2, random_state=0)
    
    n_estimators = np.arange(10, 100, step=1)
    parameters = {'n_estimators':n_estimators, 'criterion':('gini', 'entropy')}
    
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
    grid = GridSearchCV(estimator=random_forest, param_grid=parameters, n_jobs=1, cv=kfold)
    cv_results = cross_validate(grid, data, labels, scoring=scoring, cv=kfold, verbose=2)
    
    return cv_results

#cv_results = random_forests_classifier_multiple_scoring(data, labels)

# Decision tree classifier

In [ ]:
def decision_tree_classifier(train_data, train_labels, test_data, test_labels):
    decision_tree = DecisionTreeClassifier(random_state=0)
    parameters = {'criterion':('gini', 'entropy')}
    
    grid = GridSearchCV(estimator=decision_tree, param_grid=parameters, n_jobs=4)
    grid.fit(train_data, train_labels)
    print grid.best_params_
    
    test_pred = grid.predict(test_data)
    
    return metrics.accuracy_score(test_labels, test_pred)

In [ ]:
def DT_cross_validation(data, labels):
    stratified_k_fold = StratifiedKFold(n_splits=10)
    scores = []
    
    for train_index, test_index in stratified_k_fold.split(data, labels):
        train_data, test_data = data[train_index], data[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]
        score = decision_tree_classifier(train_data, train_labels, test_data, test_labels)
        print score
        scores.append(score)
        
    scores = np.array(scores)
    print np.mean(scores)
    print np.stdev(scores)
        
    return scores, np.mean(scores), np.stdev(scores)

In [ ]:
dt_scores, dt_mean, dr_stdev = DT_cross_validation(data, labels)

# {'criterion': 'entropy'}

# Multiple scores for Decision Trees

In [ ]:
def decision_tree_classifier_multiple_scoring(data, labels):
    decision_tree = DecisionTreeClassifier(random_state=0)
    parameters = {'criterion':('gini', 'entropy')}
    scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']

    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
    grid = GridSearchCV(estimator=decision_tree, param_grid=parameters, n_jobs=1, cv=kfold)
    cv_results = cross_validate(grid, data, labels, scoring=scoring, cv=kfold, verbose=2)
    
    return cv_results

#cv_results = decision_tree_classifier_multiple_scoring(data, labels)
#cv_results

# Neural network classifier

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import metrics
from keras.utils import np_utils
import sklearn.metrics as skl_metrics

def build_model(dropout_probability, input_size):
    
    model = Sequential()
    
    model.add(Dense(256, input_shape=(input_size, )))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_probability))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_probability))
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_probability))

    model.add(Dense(1, activation='sigmoid'))
   
    model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])
    
    return model
    

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

def neural_network_classifier(train_data, train_labels, test_data, test_labels):
    batch_size=256
    epochs = 50
    input_size = [train_data.shape[1]]
    neural_network = KerasClassifier(build_fn=build_model(input_size), epochs=epochs, batch_size=batch_size, verbose=0)
    
    input_size = [train_data.shape[1]]
    dropout_probability = np.arange(0.1, 0.6, step=0.1)
    parameters = {'dropout_probability':dropout_probability, 
                  'input_size': input_size}
    
    grid = GridSearchCV(estimator=neural_network, param_grid=parameters, n_jobs=1, verbose=2, cv=3)
    grid.fit(train_data, train_labels)
    print grid.best_params_
    
    test_pred = grid.predict(test_data)
    
    return skl_metrics.accuracy_score(test_labels, test_pred)

In [ ]:
def neural_network_cross_validation(data, labels):
    stratified_k_fold = StratifiedKFold(n_splits=10)
    
    scores = []
    
    for train_index, test_index in stratified_k_fold.split(data, labels):
        train_data, test_data = data[train_index], data[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]
        score = neural_network_classifier(train_data, train_labels, test_data, test_labels)
        print scores
        scores.append(score)
        
    return sum(scores)/len(scores)


In [ ]:
neural_network_cross_validation(data, labels)

# Multiple scores for Neural Network

In [ ]:
def neural_network_classifier_multiple_scoring(data, labels):
    scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']
    
    batch_size=256
    epochs = 50
    neural_network = KerasClassifier(build_fn=build_model, epochs=epochs, batch_size=batch_size, verbose=0)
    
    input_size = [data.shape[1]]
    dropout_probability = np.arange(0.1, 0.6, step=0.1)
    parameters = {'dropout_probability':dropout_probability, 
                  'input_size': input_size}
    
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
    grid = GridSearchCV(estimator=neural_network, param_grid=parameters, n_jobs=1, verbose=2, cv=kfold)

    cv_results = cross_validate(grid, data, labels, scoring=scoring, cv=kfold)
    
    return cv_results

#cv_results = neural_network_classifier_multiple_scoring(data, labels)
#cv_results

In [ ]:
cv_results

In [ ]:
for histology in hystology_types:
    results[histology] = dict()
    data = sample_genes.values
    labels = sample_primary_histology[histology].values
    
    print (histology)
    
    results[histology]['neural_network'] = neural_network_classifier_multiple_scoring(data, labels)
    results[histology]['neural_network']['histology'] = histology
    write_results_to_file('neural_network.txt', results[histology]['neural_network'])
      

# Evaluation

In [ ]:
hystology_types = sample_primary_histology.columns.values
classifiers = ['neural_network', 'random_forest', 'decision_tree', 'svm']

data = sample_genes.values
labels = sample_primary_histology['glioma'].values

results = dict()


In [ ]:
import pickle

def write_results_to_file(filename, data):
    with open(filename, 'a+b') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
num = 0
with open('neural_network_carcinoma.txt', 'rb') as handle:
    for i in range(5):
        d = pickle.load(handle)
        num = num+1
        print num
        print d

In [ ]:
def classifiers_multiple_scoring(classifier, data, labelels):
    if classifier == 'svm':
        return SVM_classifier_multiple_scoring(data, labels)
    if classifier =='random_forests':
        return random_forests_classifier_multiple_scoring(data, labels)
    if classifier == 'decision_trees':
        return decision_trees_classifier_multiple_scoring(data, labels)
    if classifier =='neural_networks':
        return neural_networks_classifier_multiple_scoaring(data, labels)
        

In [ ]:
def compute_results(classifier):
    for histology in hystology_types:
        results[histology] = dict()
        data = sample_genes.values
        labels = sample_primary_histology[histology].values

        print (histology)

        results[histology][classifier] = classifiers_multiple_scoring(classifier, data, labels)
        results[histology][classifier]['histology'] = histology
        write_results_to_file(classifier + '.txt', results[histology][classifer])

In [ ]:
classifiers = ['svm', 'random_forests', 'decision_trees', 'neural_network']
for classifier in classifiers:
    compute_results(classifier)